In [21]:
%config IPCompleter.greedy = True
%autosave 60

Autosaving every 60 seconds


## 1. NLP

### Cargar archivo

In [22]:
import pandas as pd
import os
ruta_archivo = os.path.join("imdb_dataset.csv")
df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2').sample(10, replace=False)
# df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2')
df_criticas

,Review,Label
19841,Wallace and Gromit are the main characters in ...,pos
4229,I thought maybe a film which boasted a cast in...,neg
23546,This is an exceptional film. It is part comedy...,pos
31758,"Yes, this movie is a real thief. It stole some...",neg
8878,I've watched a lot of TV through the years. So...,neg
22929,What are the movies? I mean.. what are movies ...,pos
15663,"His music, especially what we hear of it here,...",pos
14050,The brilliance of this story delivers at least...,pos
12893,I felt compelled to comment on this film becau...,pos
294,(Spoilers more than likely... nothing really i...,neg


### Palabras de parada

In [23]:
import nltk
import string
# nltk.download('punkt')
# nltk.download('stopwords')
palabras_de_parada_ingles = set(nltk.corpus.stopwords.words('english') + list(string.punctuation) + ['...'])
palabras_de_parada_ingles

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '...',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself'

### Normalizar el texto

In [24]:
#pip install pycontractions

In [25]:
import re

def limpiar_html(raw_html):
    reg_limpiar = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    texto_limpio = re.sub(reg_limpiar, '', raw_html)
    return texto_limpio

def normalizar_critica(critica):
    critica_sin_html = limpiar_html(critica)
    tokens = nltk.tokenize.casual.casual_tokenize(critica_sin_html, "english")
    tokens_normalizados = [token.lower() for token in tokens if token not in palabras_de_parada_ingles]
    return " ".join(tokens_normalizados)

def normalizar_fila(fila):
    nueva_fila = fila
    critica_normalizada = normalizar_critica(fila['Review'])
    nueva_fila['Review'] = critica_normalizada
    return nueva_fila

In [26]:
df_criticas_normalizadas = df_criticas.apply(normalizar_fila, axis=1)
df_criticas_normalizadas

,Review,Label
19841,wallace gromit main characters best cartoons e...,pos
4229,i thought maybe film boasted cast including pe...,neg
23546,this exceptional film it part comedy part dram...,pos
31758,yes movie real thief it stole shiny oscars ava...,neg
8878,i've watched lot tv years so much i start seei...,neg
22929,what movies i mean .. movies made shootings ki...,pos
15663,his music especially hear slow from around tim...,pos
14050,the brilliance story delivers least one skillf...,pos
12893,i felt compelled comment film listed fourth lo...,pos
294,spoilers likely nothing really important figur...,neg


### Obtener el vocabulario del problema

In [27]:
def obtener_vocabulario_problema(df_criticas_normalizadas):
    cuerpo_texto_normalizado = ' '.join(df_criticas_normalizadas['Review'].tolist())
    vocabulario_problema = cuerpo_texto_normalizado.split()
    vocabulario_ordenado = sorted(set(vocabulario_problema))
    return vocabulario_ordenado

def obtener_vocabulario_problema_y_posicion(vocabulario_del_problema_ordenado):
    vocabulario_y_posicion = {}
    for i, token in enumerate(vocabulario_del_problema_ordenado):
        vocabulario_y_posicion[token] = i
    return vocabulario_y_posicion

In [28]:
vocabulario_problema = obtener_vocabulario_problema_y_posicion(obtener_vocabulario_problema(df_criticas_normalizadas))
#vocabulario_problema.keys()

### One Hot Encoding

In [29]:
import numpy as np

def obtener_one_hot_vector(critica, vocabulario_problema_y_posicion):
    one_hot_vector = np.zeros(len(vocabulario_problema_y_posicion), dtype=int)
    for token in critica.split():
        one_hot_vector[vocabulario_problema_y_posicion[token]] = 1
    return one_hot_vector

In [37]:
one_hots = []
indices = []
for indice, fila in df_criticas_normalizadas.iterrows():
    one_hot = obtener_one_hot_vector(fila['Review'], vocabulario_problema)
    indices.append(indice)
    one_hots.append(one_hot)

## 2. Crear y entrenar el modelo predictivo

In [31]:
df_criticas_one_hot_vector = pd.DataFrame(columns=vocabulario_problema.keys(), data=one_hots, index=indices)
df_criticas_one_hot_vector['Target'] = df_criticas_normalizadas['Label']
X = df_criticas_one_hot_vector.drop('Target', axis=1).to_numpy()
Y = df_criticas_one_hot_vector['Target'].ravel()

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [33]:
from sklearn.linear_model import LogisticRegression
clasificador_reg_log = LogisticRegression(random_state=0, solver='liblinear') #Falta añadir el solver del magister...
clasificador_reg_log.fit(X_train, Y_train)
print('accuracy del clasificador: {0:.2f}'.format(clasificador_reg_log.score(X_test, Y_test)))

accuracy del clasificador: 1.00


### Guardar el clasificador como un archivo

In [34]:
import pickle
ruta_archivo_clasificador = os.path.join('clasificador-regresion-logistica.pkl')
archivo_clasificador = open(ruta_archivo_clasificador, 'wb')
pickle.dump(clasificador_reg_log, archivo_clasificador)
archivo_clasificador.close()

### Guardar el vocabulario del problema como un archivo

In [35]:
ruta_archivo_vocabulario = os.path.join('vocabulario-problema.pkl')
archivo_vacabulario = open(ruta_archivo_vocabulario, 'wb')
pickle.dump(vocabulario_problema, archivo_vacabulario)
archivo_vacabulario.close()

### Guardar las palabras de parada como un archivo

In [36]:
ruta_archivo_palabras_parada = os.path.join('palabras-parada.pkl')
archivo_palabras_parada = open(ruta_archivo_palabras_parada, 'wb')
pickle.dump(palabras_de_parada_ingles, archivo_palabras_parada)
archivo_palabras_parada.close()